In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import root

## Merton Model PD
The non-closed-form solution for probability of default (PD) in the Merton model is given by:

$$ E = V \Phi(d_1) - D e^{-rT}\Phi(d_2) $$
$$ \sigma_E = \frac{V}{E} \Phi(d_1)\sigma_V $$

where:
- $\Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution
- $ d_1 = \frac{\ln\frac{V}{D} + (r + 0.5{\sigma_V}^2)T}{\sigma_V\sqrt{T}} $
- $ d_2 = d_1 - \sigma_V\sqrt{T} $
- $ E $ is the equity value
- $ \sigma_E $ is the equity volatility 
- $ V $ is the asset value. This is unknown and solved for.
- $ \sigma_V $ is the asset volatility. This is unknown and solved for.
- $ D $ is the debt face value.
- $ r $ is the risk-free rate.
- $ T $ is the time to maturity.

These two functions are used to solve for $ V $ and $ \sigma_V $. Once that is done, the following equation is used to find probability of default:

$$ PD = \Phi\left( \frac{\ln\left(\frac{D}{V}\right) - \left( r - \frac{1}{2} \sigma_V^2 \right) T}{\sigma_V \sqrt{T}} \right)$$



where:

- $\Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution  
- $ V $ is the value of the firm's assets  
- $ D $ is the face value of debt 
- $ r $ is the risk-free interest rate  
- $ \sigma_V $ is the volatility of the firm's assets  
- $ T $ is the time to maturity




In [2]:
def compute_merton_pd(E, sigma_E, D, r, T=1.0, verbose=False):    

    def equations(vars):
        V, sigma_V = vars
        if V <= 0 or sigma_V <= 0:
            return 1e10, 1e10  # large penalty for invalid values
        d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
        d2 = d1 - sigma_V * np.sqrt(T)
        eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
        eq2 = (V / E) * norm.cdf(d1) * sigma_V - sigma_E
        return [eq1, eq2]

    V0 = E + D
    sigma_V0 = sigma_E
    result = root(equations, [V0, sigma_V0], method='hybr')

    if result.success:
        V_opt, sigma_V_opt = result.x
        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))
        pd = norm.cdf(-d2)
        return pd
    else:
        if verbose:
            print("Failure for firm:", E, sigma_E, D, r)
            print("Message:", result.message)
        return np.nan

In [3]:
df= pd.read_csv('../data/clean_data.csv')
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'],
        D=row['total_debt'],
        r=row['rf'] / 100,  # Convert percentage to decimal
        verbose=True
    ),
    axis=1
)
print(df)

/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_70334/2908855909.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_70334/2908855909.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))


Failure for firm: 116259732000.00002 0.1815418638389408 9448300000.0 0.0398
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 32642444760.0 0.7343823804162088 215718000000.0 0.0374
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 12798849840.0 1.184655266718994 215718000000.0 0.0347
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 5512109699.999999 1.3433913552168046 215718000000.0 0.0341
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 7232963460.0 1.3732017429642132 215718000000.0 0.0354
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 10352010900.0 1.4225119557038717 2157

In [4]:
df['pd_valid'] = df['merton_pd'].notna()
del df['SHROUT']
df.to_csv("merton_model_output.csv", index=False)

In [5]:
df['pd_valid'].value_counts()

pd_valid
True     438769
False      2281
Name: count, dtype: int64

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(df[df['merton_pd'].isna() == True])
(df[df['merton_pd'].isna() == True]).to_csv("nans.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

              date  permno   tic                          conm         PRC  \
23033   2024-10-04   44644   ADP     AUTOMATIC DATA PROCESSING  285.160000   
23772   2008-09-12   66800   AIG  AMERICAN INTERNATIONAL GROUP  203.179916   
23773   2008-09-15   66800   AIG  AMERICAN INTERNATIONAL GROUP   79.665272   
23775   2008-09-17   66800   AIG  AMERICAN INTERNATIONAL GROUP   34.309623   
23776   2008-09-18   66800   AIG  AMERICAN INTERNATIONAL GROUP   45.020921   
23777   2008-09-19   66800   AIG  AMERICAN INTERNATIONAL GROUP   64.435146   
23778   2008-09-22   66800   AIG  AMERICAN INTERNATIONAL GROUP   78.995816   
23779   2008-09-23   66800   AIG  AMERICAN INTERNATIONAL GROUP   83.682008   
23780   2008-09-24   66800   AIG  AMERICAN INTERNATIONAL GROUP   55.397490   
23781   2008-09-25   66800   AIG  AMERICAN INTERNATIONAL GROUP   50.543933   
23782   2008-09-26   66800   AIG  AMERICAN INTERNATIONAL GROUP   52.719665   
23783   2008-09-29   66800   AIG  AMERICAN INTERNATIONAL GROUP  

In [7]:
nans= pd.read_csv('nans.csv')
print(nans['leverage'].describe())
print()
print(nans['equity_volatility'].describe())

count    2281.000000
mean        0.401773
std         0.144468
min         0.017307
25%         0.271304
50%         0.377560
75%         0.544106
max         0.663637
Name: leverage, dtype: float64

count    2281.000000
mean        0.980185
std         0.594117
min         0.140918
25%         0.324153
50%         1.078506
75%         1.488964
max         2.216216
Name: equity_volatility, dtype: float64


In [8]:
print(df['leverage'].describe())
print()
print(df['equity_volatility'].describe())

count    441050.000000
mean          0.263176
std           0.163073
min           0.000000
25%           0.149472
50%           0.240519
75%           0.358133
max           1.039717
Name: leverage, dtype: float64

count    441050.000000
mean          2.206908
std          14.811601
min           0.090416
25%           0.187856
50%           0.239566
75%           0.319094
max         116.571964
Name: equity_volatility, dtype: float64
